In [1]:
import sqlalchemy
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import shap

from datetime import datetime
import pytz
from tqdm import tqdm
import pickle
import random


In [2]:
df = pd.read_csv('dataset/hack_data.csv')

In [3]:
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP']).dt.tz_localize(None)

# Get last period of data

In [4]:
# by interval
interval_day = 180
# dff = df.iloc[-interval_day:]

# full dataset
dff = df.copy()

# Preprocessing

In [5]:
me1 = [i for i in dff.columns if i.startswith('ME1')]
me1 = [i for i in me1 if not i.endswith('ECC')]

to_exclude = ['ME1_FO_FLOW_HOUR_INLET',
            'ME1_FO_DENSITY_INLET',
            'ME1_FO_TEMP_INLET',
            'ME1_FO_TOTALIZER_INLET']


me1 = [i for i in me1 if i not in to_exclude]
me2 = ['SPEED_VG','lon','lat','TIME_STAMP']


df2=pd.concat([dff[me1], dff[me2]], axis=1)
df2 = df2.loc[:,~df2 .columns.duplicated()]

# Filter Value
df3=df2[(df2['ME1_RPM']>0) & (df2['ME1_SCAV_AIR_PRESS']>0) & (df2['ME1_LO_INLET_PRESS']>0) & (df2['ME1_FOC']>0)]
df3=df3[(df3['ME1_RPM']>df3['ME1_RPM'].describe()['75%']) & (df3['SPEED_VG'] > 14)]

# Aggregating Cylinders Variable (Averaging)
pco=[i for i in df3.columns if 'PCO' in i]
cfw=[i for i in df3.columns if 'CFW' in i]
exh=[i for i in df3.columns if 'EXH' in i]
df3['ME1_CYL_PCO_OUTLET_TEMP']=df3[pco].mean(axis=1)
df3['ME1_CYL_CFW_OUTLET_TEMP']=df3[cfw].mean(axis=1)
df3['ME1_CYL_EXH_GAS_OUTLET_TEMP']=df3[exh].mean(axis=1)
df3 = df3.drop(pco+cfw+exh,axis=1)

df4 = df3.drop(['SPEED_VG','lon','lat','TIME_STAMP'],axis=1).copy()

# Isolation Forest

In [6]:
iso = IsolationForest(random_state=4)
iso.fit(df4)
iso_score=iso.score_samples(df4)

# SHAP 

In [7]:
# SHAP Explainer
explainer = shap.TreeExplainer(iso,data=df4)
#Calculate shap with shap_values method
shap_values = explainer.shap_values(df4,check_additivity=True)
dfShap = pd.DataFrame(shap_values,columns=df4.columns,index=df4.index)

dfShap_temp = dfShap[:].copy()
dfShap_temp['ISO_SCORE'] = iso_score

 95%|=================== | 2108/2222 [00:17<00:00]       

# Automatic Clustering

In [8]:
MIN_CLUSTER = 2
MAX_CLUSTER = 8

In [9]:
fixed_th = np.quantile(dfShap_temp['ISO_SCORE'],0.075)
dfShap_temp1 = dfShap_temp[dfShap_temp['ISO_SCORE'] <= fixed_th].drop(['ISO_SCORE'],axis=1)
dfShap_temp2 = dfShap_temp[dfShap_temp['ISO_SCORE'] > fixed_th].drop(['ISO_SCORE'],axis=1)
        

def auto_cluster(df_shap, min_cluster, max_cluster):
    num_class = 0
    sil_score = 0

    if len(df_shap) >= 3 :
        for num_cluss in tqdm(range(min_cluster,max_cluster+1)):
            kms_mod = KMeans(n_clusters=num_cluss,random_state=42)
            labels_test = kms_mod.fit_predict(df_shap)
            centroids_test = kms_mod.cluster_centers_
            silhouette_avg = silhouette_score(df_shap, labels_test)
            if silhouette_avg > sil_score :
                sil_score = silhouette_avg
                num_class = num_cluss
            else :
                pass 
    else :
        num_class = 1   

    # ---- Start Model with Best Num Clus
    kms_mod = KMeans(n_clusters=num_class,random_state=42)
    labels_test = kms_mod.fit_predict(df_shap)
    centroids_test = kms_mod.cluster_centers_

    return labels_test, centroids_test, kms_mod

labels_cluster, centroids_cluster, model_cluster =  auto_cluster(dfShap_temp1, MIN_CLUSTER, MAX_CLUSTER)

dfShap_temp1['CLUSTER'] = labels_cluster
dfShap_temp2['CLUSTER'] = np.nan

dfShap = pd.concat([dfShap_temp1, dfShap_temp2])

100%|██████████| 7/7 [00:00<00:00, 33.39it/s]


# Datamart

In [10]:
# datamart creation
datamart = df3.join(dfShap, rsuffix='_SHAP') 
datamart['ISO_SCORE'] = iso_score
datamart = datamart[datamart.columns.difference(df2.columns)]


datamart = df2.join(datamart)
datamart['INGESTION_DT'] = dff['TIME_STAMP']

In [11]:
datamart.head()

,ME1_RPM,ME1_RPM_SHAFT,ME1_SHAFT_POWER,ME1_SHAFT_TORQUE,ME1_SHAFT_THRUST,ME1_FO_INLET_PRESS,ME1_FO_INLET_TEMP,ME1_JCW_INLET_TEMP,ME1_JCW_INLET_PRESS,ME1_CYL1_CFW_OUTLET_TEMP,...,ME1_SHAFT_THRUST_SHAP,ME1_SHAFT_TORQUE_SHAP,ME1_TC1_LO_INLET_PRESS_SHAP,ME1_TC1_LO_OUTLET_TEMP_SHAP,ME1_TC1_RPM_SHAP,ME1_TC2_LO_INLET_PRESS_SHAP,ME1_TC2_LO_OUTLET_TEMP_SHAP,ME1_TC2_RPM_SHAP,ME1_THRUST_PAD_TEMP_SHAP,INGESTION_DT
0,51.060,50.9,0.0,0.0,2220.0,7.686,134.763,78.115,4.352,119.036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:00:00
1,50.970,50.9,0.0,0.0,2220.0,7.687,133.605,77.957,4.340,119.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:10:00
2,51.120,50.9,0.0,0.0,2220.0,7.697,135.074,78.249,4.356,119.076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:20:00
3,54.042,50.9,0.0,0.0,2220.0,7.637,135.089,77.434,4.355,118.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:30:00
4,56.376,50.9,0.0,0.0,2220.0,7.593,135.735,77.107,4.355,117.144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:40:00


In [1]:
!tar cvfz all_files_hackathon_maritime.tar.gz *

1) development short pred (lstm)/
1) development short pred (lstm)/development.ipynb
1) development short pred (lstm)/baseline_normalization.csv
1) development short pred (lstm)/isolation_forest.pkl
1) development short pred (lstm)/model_LSTM.h5
1) development short pred (lstm)/.ipynb_checkpoints/
1) development short pred (lstm)/.ipynb_checkpoints/development-checkpoint.ipynb
1) development short pred (lstm)/1-LSTMbackend.py
20211121 - fbprophet.ipynb
20221110_0 - Time generator preprocessing.ipynb
20221110_1 - safety index anomaly process.ipynb
20221110_2 - LSTM forecasting.ipynb
20221121_LSTM_agung.ipynb
2) development long pred (prophet)/
2) development long pred (prophet)/2-prophetbackend.py
2) development long pred (prophet)/model/
2) development long pred (prophet)/model/fb_prophet.sav
2) development long pred (prophet)/20221122 - development fbprophet.ipynb
2) development long pred (prophet)/.ipynb_checkpoints/
2) development long pred (prophet)/.ipynb_checkpoints/20221122 - de

mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/__pycache__/concurrent.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/__pycache__/log.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/__init__.py
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/__pycache__/
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/__pycache__/posix.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/__pycache__/common.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/__pycache__/interface.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq/eventloop/minitornado/platform/__pycache__/auto.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/zmq

mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_lexers.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_deepreload.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_editorhooks.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_pretty.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_display.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_imports.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_backgroundjobs.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__/test_latextools.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/lib/tests/__pycache__

mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/test_prompts.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/print_argv.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/test_pylabtools.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/test_displayhook.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/test_magic.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/nonascii.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/test_handlers.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/__pycache__/test_oinspect.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/core/tests/test_events.py
mga_hack/lib/python3.8/site-packages/IPython/core/tests/test_magic.p

mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/test_path.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/test_dir2.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/test_decorators.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/test_tempdir.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/test_importstring.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/__pycache__/test_wildcard.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/test_tempdir.py
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/test_tokenutil.py
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/test_shimmodule.py
mga_hack/lib/python3.8/site-packages/IPython/utils/tests/test_capture.py
mga_hack/lib/python3.8/si

...

mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/__pycache__/etcd_rendezvous_backend.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/__pycache__/c10d_rendezvous_backend.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/__pycache__/etcd_store.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/__pycache__/registry.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/__pycache__/utils.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/etcd_store.py
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/dynamic_rendezvous.py
mga_hack/lib/python3.8/site-packages/torch/distributed/elastic/rendezvous/c10d_rendezvous_backend.py
mga_hack/lib/python3.8/sit

mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/_fsdp_extensions.py
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/fully_sharded_data_parallel.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/sharded_grad_scaler.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/_optim_utils.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/flat_param.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/_symbolic_trace.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/flatten_params_wrapper.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/_utils.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/distributed/fsdp/__pycache__/_fsdp_extensions.cpython-38.pyc
mga_hack/lib/pyt

mga_hack/lib/python3.8/site-packages/torch/fx/experimental/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/__pycache__/meta_tracer.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/__pycache__/schema_type_annotation.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/__pycache__/partitioner_utils.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/__pycache__/refinement_types.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/merge_matmul.py
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/migrate_gradual_types/
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/migrate_gradual_types/constraint_transformation.py
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/migrate_gradual_types/__init__.py
mga_hack/lib/python3.8/site-packages/torch/fx/experimental/migrate_gradual_types/constraint.py
mga_hack/lib/python3.8/site-packages

mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/_stdlib.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/_mock.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/importer.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/_directory_reader.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/_digraph.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/__init__.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/__pycache__/_importlib.cpython-38.pyc
mga_hack/lib/python3.8/site-packages/torch/package/_mock.py
mga_hack/lib/python3.8/site-packages/torch/package/_package_pickler.py
mga_hack/lib/python3.8/site-packages/torch/package/find_file_dependencies.py
mga_hack/lib/python3.8/site-packages/torch/package/_directory_reader.py
mga_hack/lib/python3.8/site-packages/torch/package/package_importer.py
mga_hack/lib/python3.8/site-packa

mga_hack/lib/python3.8/site-packages/torch/lib/libtorch_cuda.so
mga_hack/lib/python3.8/site-packages/torch/lib/libtorch.so
mga_hack/lib/python3.8/site-packages/torch/lib/libc10.so
mga_hack/lib/python3.8/site-packages/torch/lib/libgomp-a34b3233.so.1
mga_hack/lib/python3.8/site-packages/torch/lib/libcudart-e409450e.so.11.0
mga_hack/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so
^C
